In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import dataset
df = pd.read_csv('./dataset/230322_OlderPredictTc_data_thermal.csv')

# Only use previous values from same individual
df['previous_tre_int'] = df.groupby('id_all')['tre_int'].shift(1)
df['previous_mtsk_int'] = df.groupby('id_all')['mtsk_int'].shift(1)

# Select only time > 0
df = df[df.time > 0]

# Select only features and output
features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set', 'previous_tre_int', 'previous_mtsk_int']
output = ['tre_int', 'mtsk_int']
df = df[features + output + ['id_all', 'study', 'condition', 'time']]

# Create train_df based on participants assigned to training set
train_ids = [46, 34, 68, 30, 40, 98, 89, 65, 24, 58, 85, 67, 28, 39, 35, 77, 26,
             80, 70, 37, 52, 56, 74, 78, 71, 60, 86, 43, 91, 82, 22, 59, 21, 87,
             95, 66, 44, 25, 76, 94, 53, 32, 73, 23, 49]
train_df = df[df['id_all'].isin(train_ids)]

# Check data
train_df.isnull().sum()

In [ ]:
# TRAIN
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit, GroupShuffleSplit

features_scaler = MinMaxScaler(feature_range=(0,1))
output_scaler = MinMaxScaler(feature_range=(0,1))

# Fit scalers
train_features = features_scaler.fit_transform(train_df[features])
train_output = output_scaler.fit_transform(train_df[output])

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train_features, train_output)

In [ ]:
import numpy as np

# Setting precision to maximum
np.set_printoptions(precision=50)

# Your code
coefficients = model.coef_
intercept = model.intercept_

print(coefficients)
print(intercept)

In [ ]:
# Sample coefficients and intercept for demonstration
coefficients_tre = [1.0646e-04, -4.6883e-05, -3.9064e-04, 2.3522e-04, 8.7962e-04, 5.0335e-04, 9.9441e-01, 4.6675e-03]
intercept_tre = -2.7589e-04

coefficients_mtsk = [7.7116e-04, 7.1947e-04, -5.1858e-04, 3.4974e-04, 1.4747e-02, 2.0432e-03, -9.6831e-04, 9.4203e-01]
intercept_mtsk = 3.9924e-02

# List of input features
features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set', 'previous_tre_int', 'previous_mtsk_int']

# Generate LaTeX equations
equations = []
for coef, output_name in zip(coefficients, ['tre\\_int', 'mtsk\\_int']):
    equation = "{} = ".format(output_name)
    for i, c in enumerate(coef):
        equation += "{:.2e} x_{{{}}} + ".format(c, i+1)
    equation += "{:.2e}".format(intercept[0] if output_name == 'tre\\_int' else intercept[1])
    equations.append(equation)

# Print LaTeX equations
for eq in equations:
    print("${}$".format(eq))
print("\n\\begin{itemize}")
for i, feature in enumerate(features):
    print("  \\item $x_{{{}}}$: {}".format(i+1, feature))
print("\\end{itemize}")


In [ ]:
# SIMULATE
from helpers import get_sample

# Simulate initial 60 minutes
def simulate_initial(row):
    # Initial ambient conditions for cooling environment (22°C 9% RH)
    initial_ambient = [22, 9]
    # Initial core and skin temp
    body_conditions = [37, 32]
    # Body parameters taken from data
    body_parameters = row[features[:-4]].tolist()
    # Simulate for 60 mins
    for i in range(60):
        X = np.array(body_parameters + initial_ambient + body_conditions).reshape(1, -1)
        X = features_scaler.transform(X)
        y = model.predict(X)
        y = output_scaler.inverse_transform(y)
        body_conditions = y[0].tolist()
    # Return core and skin temp at end of 60 mins (e.g. [37, 32])
    return body_conditions


def run_and_save_trial(study, condition):
    # Get sample
    sample = get_sample(study, condition)
    # Group by unique id
    grouped = sample.groupby('id_all', sort=False)
    all_predicted_values = []
    for name, group in grouped:
        predicted_values = False
        for index, row in group.iterrows():
            # Get initial conditions
            if not predicted_values:
                predicted_values = [simulate_initial(row)]
                continue
            # Calculate next temp values
            X = np.array(row[features[:-2]].tolist() + predicted_values[-1]).reshape(1, -1)
            X = features_scaler.transform(X)
            y = model.predict(X)
            y = output_scaler.inverse_transform(y)
            predicted_values.append(y[0].tolist())
        all_predicted_values += predicted_values

    all_core_temps = np.array(all_predicted_values)[:, 0]
    all_skin_temps = np.array(all_predicted_values)[:, 1]
    print(all_core_temps.shape[0])
    print(all_skin_temps.shape[0])
    print(sample.shape[0])
    # Save to csv
    df = pd.DataFrame(all_core_temps, columns=["tre_predicted"])
    df["mtsk_predicted"] = all_skin_temps
    df.to_csv('results/regression-{}-{}.csv'.format(study, condition), index=False)

In [ ]:
run_and_save_trial('heatwave 1 (prolonged)', 'hot')
run_and_save_trial('heatwave 2 (indoor)', 'cool')
run_and_save_trial('heatwave 2 (indoor)', 'temp')
run_and_save_trial('heatwave 2 (indoor)', 'warm')
run_and_save_trial('heatwave 2 (indoor)', 'hot')
run_and_save_trial('heatwave 3 (cooling)', 'hot')